In [16]:
from feature_engineering import apply_feature_engineering, add_dummies
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
def load_data():
    """
    Load the dataset using parquet and pyarrow
    """
    df = pd.read_parquet(
        "itineraries_snappy.parquet", 
        engine= "pyarrow", 
        columns= [
            "searchDate", 
            "flightDate", 
            "startingAirport", 
            "destinationAirport",
            "travelDuration", 
            "isBasicEconomy", 
            "isRefundable", 
            "isNonStop", 
            "totalFare", 
            "seatsRemaining", 
            "totalTravelDistance",
            "segmentsDepartureTimeRaw", 
            "segmentsAirlineCode", 
            "segmentsCabinCode"
        ]
    )
    return df

In [3]:
df = load_data()
df.head()

,searchDate,flightDate,startingAirport,destinationAirport,travelDuration,isBasicEconomy,isRefundable,isNonStop,totalFare,seatsRemaining,totalTravelDistance,segmentsDepartureTimeRaw,segmentsAirlineCode,segmentsCabinCode
0,2022-04-16,2022-04-17,ATL,BOS,PT2H29M,False,False,True,248.6,9,947.0,2022-04-17T12:57:00.000-04:00,DL,coach
1,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,False,True,248.6,4,947.0,2022-04-17T06:30:00.000-04:00,DL,coach
2,2022-04-16,2022-04-17,ATL,BOS,PT2H30M,False,False,True,248.6,9,947.0,2022-04-17T11:35:00.000-04:00,DL,coach
3,2022-04-16,2022-04-17,ATL,BOS,PT2H32M,False,False,True,248.6,8,947.0,2022-04-17T13:59:00.000-04:00,DL,coach
4,2022-04-16,2022-04-17,ATL,BOS,PT2H34M,False,False,True,248.6,9,947.0,2022-04-17T09:59:00.000-04:00,DL,coach


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82138753 entries, 0 to 82138752
Data columns (total 14 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   searchDate                object 
 1   flightDate                object 
 2   startingAirport           object 
 3   destinationAirport        object 
 4   travelDuration            object 
 5   isBasicEconomy            bool   
 6   isRefundable              bool   
 7   isNonStop                 bool   
 8   totalFare                 float64
 9   seatsRemaining            int64  
 10  totalTravelDistance       float64
 11  segmentsDepartureTimeRaw  object 
 12  segmentsAirlineCode       object 
 13  segmentsCabinCode         object 
dtypes: bool(3), float64(2), int64(1), object(8)
memory usage: 7.0+ GB


In [6]:
def remove_expensive_flights(df):
    """
    Grouping flights by date, departure, and destination airports, 
    then removing the most expensive flight from each group.
    
    Returns:
    pandas.DataFrame: DataFrame with most expensive flights removed
    """
    # Copy of the dataframe
    df_processed = df.copy()
    
    # Grouping by date and departure and arrival airports (route)
    groupby_cols = ['flightDate', 'startingAirport', 'destinationAirport']
    
    # Identify the most expensive flight
    idx_to_drop = df_processed.groupby(groupby_cols)['totalFare'].idxmax()
    
    # Dropping the most expensive flights
    df_processed = df_processed.drop(index=idx_to_drop)
    
    print(f"Removed {len(idx_to_drop)} expensive flights out of {len(df)} total flights")
    print(f"Remaining flights: {len(df_processed)}")
    
    return df_processed

In [7]:
df_cheaper = remove_expensive_flights(df)

Removed 50759 expensive flights out of 82138753 total flights
Remaining flights: 82087994


In [8]:
# Checking after processing
example_route_after = df_cheaper[(df_cheaper['flightDate'] == '2022-04-17') & 
                                 (df_cheaper['startingAirport'] == 'ATL') & 
                                 (df_cheaper['destinationAirport'] == 'BOS')]
print("\nExample route after processing:")
print(example_route_after[['flightDate', 'startingAirport', 'destinationAirport', 'totalFare']].sort_values('totalFare'))

# Specific route before processing
example_route = df[(df['flightDate'] == '2022-04-17') & 
                   (df['startingAirport'] == 'ATL') & 
                   (df['destinationAirport'] == 'BOS')]
print("\nExample route before processing:")
print(example_route[['flightDate', 'startingAirport', 'destinationAirport', 'totalFare']].sort_values('totalFare'))

# Verifying the most expensive flight was removed
max_fare_before = example_route['totalFare'].max()
max_fare_after = example_route_after['totalFare'].max()
print(f"\nMax fare before: ${max_fare_before}")
print(f"Max fare after: ${max_fare_after}")
print(f"Was most expensive removed? {'Yes' if max_fare_before > max_fare_after else 'No'}")


Example route after processing:
    flightDate startingAirport destinationAirport  totalFare
0   2022-04-17             ATL                BOS     248.60
1   2022-04-17             ATL                BOS     248.60
2   2022-04-17             ATL                BOS     248.60
3   2022-04-17             ATL                BOS     248.60
4   2022-04-17             ATL                BOS     248.60
5   2022-04-17             ATL                BOS     248.60
8   2022-04-17             ATL                BOS     251.10
9   2022-04-17             ATL                BOS     251.10
6   2022-04-17             ATL                BOS     251.10
7   2022-04-17             ATL                BOS     251.10
10  2022-04-17             ATL                BOS     252.60
11  2022-04-17             ATL                BOS     252.60
12  2022-04-17             ATL                BOS     252.60
13  2022-04-17             ATL                BOS     252.60
14  2022-04-17             ATL                BOS   

In [12]:
def keep_cheapest_flights(df):
    """
    Group flights by date, departure, and destination airports, 
    then keep only the cheapest flight from each group.
    """
    # Copy of the dataframe
    df_processed = df.copy()
    
    # Grouping by date and route 
    groupby_cols = ['flightDate', 'startingAirport', 'destinationAirport']
    
    # Identifying the cheapest flight
    idx_to_keep = df_processed.groupby(groupby_cols)['totalFare'].idxmin()
    
    # Keeping only the cheapest flights
    df_processed = df_processed.loc[idx_to_keep]
    
    print(f"Kept {len(df_processed)} cheapest flights out of {len(df)} total flights")
    print(f"Removed {len(df) - len(df_processed)} more expensive flights")
    
    return df_processed

In [13]:
def verify_cheapest_flights(original_df, processed_df):
    """
    Verify that the processed dataframe contains only the cheapest flight 
    for each unique route and date combination.
    """
    # Grouping
    groupby_cols = ['flightDate', 'startingAirport', 'destinationAirport']
    
    # Counting groups in original and processed dataframes
    original_groups = original_df.groupby(groupby_cols).ngroups
    processed_groups = processed_df.groupby(groupby_cols).ngroups
    
    # Number of groups should be the same in both dataframes
    if original_groups != processed_groups:
        print(f"ERROR: Number of route-date combinations doesn't match.")
        print(f"Original: {original_groups}, Processed: {processed_groups}")
        return False
    
    # Checking if each group in processed contains the cheapest flight from the original
    verification_success = True
    
    # Sample a subset
    sample_size = min(100, original_groups)
    sampled_groups = original_df.groupby(groupby_cols).apply(
        lambda x: x.name).sample(n=sample_size).tolist() if sample_size < original_groups else original_df.groupby(groupby_cols).groups.keys()
    
    for group in sampled_groups:
        if isinstance(group, tuple):
            group_key = group
        else:
            group_key = tuple(group)
        
        # Filter condition
        filter_conditions = True
        for i, col in enumerate(groupby_cols):
            filter_conditions = filter_conditions & (original_df[col] == group_key[i])
            
        # Getting min fare from original for this group
        original_min_fare = original_df[filter_conditions]['totalFare'].min()
        
        # Checking if this group exists in processed and has the expected min fare
        processed_group = processed_df
        for i, col in enumerate(groupby_cols):
            processed_group = processed_group[processed_group[col] == group_key[i]]
        
        if len(processed_group) == 0:
            print(f"ERROR: Group {group_key} missing from processed dataframe")
            verification_success = False
            continue
            
        processed_fare = processed_group['totalFare'].values[0]
        
        if abs(processed_fare - original_min_fare) > 0.01:  # Allowing for small floating point differences
            print(f"ERROR: For group {group_key}, processed fare ${processed_fare} doesn't match original min fare ${original_min_fare}")
            verification_success = False
    
    # Checking that each route-date combination appears exactly once in processed
    duplicate_groups = processed_df.groupby(groupby_cols).size()
    has_duplicates = (duplicate_groups > 1).any()
    
    if has_duplicates:
        duplicate_count = (duplicate_groups > 1).sum()
        print(f"ERROR: Found {duplicate_count} route-date combinations with multiple flights in processed data")
        print("Example duplicates:")
        print(duplicate_groups[duplicate_groups > 1].head())
        verification_success = False
    

    if verification_success:
        print(f"VERIFICATION PASSED: All sampled route-date combinations ({sample_size}/{original_groups}) have the cheapest flight from the original dataset")
        print(f"Original dataset: {len(original_df)} flights")
        print(f"Processed dataset: {len(processed_df)} flights")
        print(f"Reduction: {((len(original_df) - len(processed_df)) / len(original_df) * 100):.2f}%")
        
        # Few examples for manual verification
        print("\nSample verification (3 random groups):")
        random_groups = list(processed_df.groupby(groupby_cols).groups.keys())
        import random
        random.shuffle(random_groups)
        
        for i, group in enumerate(random_groups[:3]):
            group_filter = True
            for j, col in enumerate(groupby_cols):
                group_filter = group_filter & (original_df[col] == group[j])
                
            original_fares = original_df[group_filter]['totalFare'].sort_values()
            processed_fare = processed_df[group_filter]['totalFare'].values[0]
            
            print(f"Group {i+1}: {group}")
            print(f"  Original fares: {', '.join(['$' + str(fare) for fare in original_fares.values])}")
            print(f"  Cheapest fare: ${original_fares.min()}")
            print(f"  Processed fare: ${processed_fare}")
            print(f"  Correctly kept cheapest: {abs(processed_fare - original_fares.min()) < 0.01}")
        
    return verification_success

In [14]:
# Keeping only the cheapest flights
df_cheapest = keep_cheapest_flights(df)

verification_result = verify_cheapest_flights(df, df_cheapest)

Kept 50759 cheapest flights out of 82138753 total flights
Removed 82087994 more expensive flights
VERIFICATION PASSED: All sampled route-date combinations (100/50759) have the cheapest flight from the original dataset
Original dataset: 82138753 flights
Processed dataset: 50759 flights
Reduction: 99.94%

Sample verification (3 random groups):
Group 1: ('2022-09-28', 'CLT', 'DTW')
  Original fares: $80.08, $91.08, $91.08, $91.08, $91.08, $91.08, $91.08, $91.08, $91.08, $91.08, $91.08, $91.08, $91.08, $91.08, $91.08, $91.08, $91.08, $91.08, $91.08, $91.08, $91.08, $91.08, $91.08, $91.08, $91.08, $92.1, $92.1, $92.1, $92.1, $92.1, $92.1, $92.1, $92.1, $92.1, $92.1, $92.1, $92.1, $92.1, $92.1, $92.1, $92.1, $92.1, $92.1, $92.1, $92.1, $92.1, $97.08, $97.08, $97.08, $97.08, $97.08, $97.08, $97.08, $97.08, $97.08, $97.08, $97.08, $97.08, $97.08, $99.08, $99.08, $99.08, $99.08, $99.08, $99.08, $99.08, $102.1, $102.1, $102.1, $102.1, $102.1, $102.1, $102.1, $102.1, $102.1, $102.1, $102.1, $102.

In [15]:
# Check for 2022-09-28 CLT to DTW original and processed
print("\nOriginal:")
print(df[(df['flightDate'] == '2022-09-28') & 
         (df['startingAirport'] == 'CLT') & 
         (df['destinationAirport'] == 'DTW')].sort_values('totalFare')[['totalFare']])
print("\nProcessed:")
print(df_cheapest[(df_cheapest['flightDate'] == '2022-09-28') & 
                  (df_cheapest['startingAirport'] == 'CLT') & 
                  (df_cheapest['destinationAirport'] == 'DTW')].sort_values('totalFare')[['totalFare']])
print("\nVerification result:", verification_result)


Original:
          totalFare
52461605      80.08
53827520      91.08
53827521      91.08
56824939      91.08
57969990      91.08
...             ...
73896518     605.70
74426392     605.70
74876086     605.70
73367097     605.70
75441372     605.70

[2701 rows x 1 columns]

Processed:
          totalFare
52461605      80.08

Verification result: True


# Adding new columns

In [17]:


def add_historical_fares(df, days_lookback=7):
    """
    Adding historical fare columns (t-1, t-2, ..., t-days_lookback) to the dataframe.
    Each column contains the totalFare from n days before the current searchDate
    for the same route.
    """
    # Ccopy of the dataframe
    df_with_history = df.copy()
    
    # Making sure the searchDate is in datetime format
    if df_with_history['searchDate'].dtype == 'object':
        df_with_history['searchDate'] = pd.to_datetime(df_with_history['searchDate'])
    
    fare_lookup = {}
    
    # Lookup dictionary
    print("Building fare lookup dictionary...")
    for _, row in df.iterrows():
        key = (row['flightDate'], row['startingAirport'], row['destinationAirport'], row['searchDate'])
        fare_lookup[key] = row['totalFare']
    
    # Adding columns for the t-n fares
    for days_back in range(1, days_lookback + 1):
        col_name = f't-{days_back}'
        print(f"Adding {col_name} column...")
        
        # Initiate with NaN
        df_with_history[col_name] = np.nan
        
        # For each row, look up the fare from n days before
        for idx, row in df_with_history.iterrows():
            prev_search_date = (pd.to_datetime(row['searchDate']) - timedelta(days=days_back)).strftime('%Y-%m-%d')
            
            # Trying to find the fare for the same route on the previous search date
            lookup_key = (row['flightDate'], row['startingAirport'], row['destinationAirport'], prev_search_date)
            
            if lookup_key in fare_lookup:
                df_with_history.at[idx, col_name] = fare_lookup[lookup_key]
    
    return df_with_history

In [18]:
df_with_history = add_historical_fares(df, days_lookback=7)
df_with_history.head()

Building fare lookup dictionary...
Adding t-1 column...
Adding t-2 column...
Adding t-3 column...
Adding t-4 column...
Adding t-5 column...
Adding t-6 column...


: 